In [22]:
import wget
url = 'https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz'
filename = wget.download(url)
print(filename)

lenta-ru-news.csv (1).gz


In [23]:
from corus import load_lenta

path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)
next(records)

LentaRecord(
    url='https://lenta.ru/news/2018/12/14/cancer/',
    title='Названы регионы России с\xa0самой высокой смертностью от\xa0рака',
    text='Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
    topic='Россия',
    tags='Общество',
    date=None
)

In [24]:
!pip install pymorphy2 stop-words annoy

In [26]:
!pip install tensorflow

  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)


In [39]:
import pandas as pd
import numpy as np

import gensim.downloader as api
from gensim.models import Word2Vec, FastText
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import string
from sklearn.model_selection import train_test_split
import annoy
import re
from tqdm import tqdm
from tqdm import tqdm_notebook

import gensim.models
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [40]:
# Сформируем корпус текстов

# возьмем заголовок текста
corpus_title = []
# возьмем сам текст
corpus_text = []

for num, iter in tqdm(enumerate(records)):
    corpus_title.append(iter.title)
    corpus_text.append(iter.text)
    if num == 99999:
        break

99999it [00:02, 42256.52it/s]


In [41]:
corpus_title[10], corpus_text[10]

('Пензенский губернатор решил оставить без премий работающих сотрудников',
 'Губернатор Пензенской области Иван Белозерцев предложил наказывать сотрудников, которые выходят на работу, несмотря на болезнь. Об этом сообщает издание PenzaInform в понедельник, 9 января. «Если руководитель видит, что человек пришел на работу больным, его надо наказать, лишить премии. У нас же люди думают, что они незаменимы, и идут совершать подвиги», — сказал он на совещании регионального правительства. По мнению главы региона, необходимо менять психологию как рядовых сотрудников, которые выходят на работу с повышенной температурой во время эпидемии гриппа и заражают коллег, так и их начальников, которые допускают распространение вируса. По информации НИИ гриппа, на конец декабря прошлого года в Пензе эпидемический порог был превышен примерно на 20 процентов. В октябре Роспотребнадзор рекомендовал работодателям измерять температуру сотрудникам, чтобы избежать распространения этого заболевания.')

In [42]:
df = pd.DataFrame(corpus_title, columns=['title'])
df['text'] = corpus_text
df.head()

,title,text
0,Назван самый популярный российский телеканал 2...,В 2016 году Первый канал уступил первое место ...
1,Музыкант исполнил песню из «Титаника» ноздрями,Музыкант Сайрус Бхиладвала (Cyrus Bhiladvala) ...
2,Преподаватель покончил с собой в аудитории пет...,Управление Следственного комитета по Санкт-Пет...
3,В Москве построят стеклянный автовокзал,На месте здания автовокзала «Щелковский» на во...
4,Асад заявил о готовности обсуждать любые вопро...,Президент Сирии Башар Асад заявил о готовности...


In [43]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [44]:
#1. Функция преобразования (лемматизация, удаление стоп-слов и пунктуации)
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [45]:
pattern = r'[^а-яА-Я0-9]'
df['text'] = df['text'].apply(lambda x: re.sub(pattern, ' ', x))

In [46]:
# создание списка токенов
%time

sentences = []
counter = 0

for line in tqdm_notebook(df.text):
    spls = preprocess_txt(line)
    sentences.append(spls)
    counter += 1      
    if counter > 100000:
        break

CPU times: total: 0 ns
Wall time: 0 ns


  0%|          | 0/100000 [00:00<?, ?it/s]

In [47]:

sentences[0]

['2016',
 'канал',
 'уступить',
 'место',
 'число',
 'телезритель',
 'россия',
 '1',
 'писать',
 'газета',
 'ведомость',
 'ссылка',
 'дать',
 'бывший',
 'данные',
 'телеизмерительный',
 'компания',
 'канал',
 'вгтрк',
 'среднее',
 'смотреть',
 '12',
 '9',
 'процент',
 'россиянин',
 'возраст',
 'старший',
 'показатель',
 'составить',
 '12',
 '7',
 'процент',
 'слово',
 'представитель',
 'полный',
 'текст',
 'исследование',
 'иметься',
 'распоряжение',
 'телеканал',
 '2015',
 'россия',
 '1',
 'смотреть',
 '12',
 '7',
 'процент',
 'зритель',
 'канал',
 '13',
 '7',
 'процент',
 'терять',
 'лидерский',
 'позиция',
 'аудитория',
 '2012',
 'уступить',
 'место',
 'нтв',
 'последний',
 'усилить',
 'сеть',
 'региональный',
 'новость',
 'запустить',
 'востребовать',
 'развлекательный',
 'формат',
 'выходной',
 'например',
 'удивительный',
 'всероссийский',
 'открытый',
 'конкурс',
 'детский',
 'талант',
 'синий',
 'птица',
 'канал',
 'смочь',
 'усилить',
 'сериальный',
 'направление',
 'будни',
 

In [48]:
%time
modelW2V = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=2)

CPU times: total: 0 ns
Wall time: 0 ns


In [49]:
%time
modelFT = FastText(sentences=sentences, vector_size=100, min_count=2, window=5, workers=8)

CPU times: total: 0 ns
Wall time: 0 ns


In [50]:
w2v_index = annoy.AnnoyIndex(100 ,'angular')
ft_index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

for line in tqdm_notebook(df.text):
        n_w2v = 0
        n_ft = 0
        spls = line
        index_map[counter] = spls
        question = preprocess_txt(spls)
        
        vector_w2v = np.zeros(100)
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]
                n_w2v += 1
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        # ограничение в  100000 строк
        if counter > 100000:
            break
    

w2v_index.build(10)
ft_index.build(10)

  0%|          | 0/100000 [00:00<?, ?it/s]

True

In [51]:
# 5 ближайших

def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
#     print(question)
    vector = np.zeros(100)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, )
    return [index_map[i] for i in answers]

In [52]:
TEXT = 'экономическая политика России'

In [53]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['Россия готова вместе с Евросоюзом формировать  большую Европу   несмотря на всю ошибочность санкций и ответственность Евросоюза за кризис на Украине  Такое заявление сделал вице премьер Игорь Шувалов  выступая на заседании Валдайского клуба в пятницу  19 июня  в рамках Петербургского международного экономического форума  ПМЭФ   передает  Интерфакс    Мы все время поддерживали эту идею  большой Европы   формирования общего экономического пространства  И понимали  что затянувшаяся дискуссия по новому базовому договору  между РФ и ЕС    это большая ошибка     сказал Шувалов  Он подчеркнул  что  несмотря на введенные Европой в отношении России санкции  в правительстве не забыли о необходимости укрепления европейского сотрудничества и о формировании с Евросоюзом общего экономического пространства  Вице премьер выразил надежду  что после разрешения украинского кризиса Россия и Брюссель смогут построить прочные партнерские отношения  Также Шувалов подчеркнул  что причиной кризиса на Украине

In [54]:
get_response(TEXT, ft_index, modelFT, index_map)

['Канцлер Германии Ангела Меркель заявила о важности создания зоны свободной торговли с Россией  передает ТАСС  Такую точку зрения она высказала на экономическом форуме в городе Штральзунде   Перспектива большой зоны свободной торговли   Россией важна и для нас     сказала Меркель  Она отметила  что между двумя странами   есть проблемы политического характера   но  по ее мнению  их можно преодолеть путем политического диалога  Меркель подчеркнула  что соглашения  об ассоциации с ЕС   которые были заключены с Молдавией и Украиной  не направлены против России  а  напротив  двигают страны к общей торговой зоне   Мы приближаем их шаг за шагом к общему экономическому пространству  как однажды сказал  президент РФ  Владимир Путин   от Владивостока до Лиссабона     сказала Меркель  Кроме того  канцлер ФРГ отметила важность выполнения минских соглашений  О том  что между ЕС и ЕАЭС может быть создана зона свободной торговли  впервые было заявлено в конце 2014 года  Меркель обосновала это  в час

In [55]:
TEXT = 'экономика'

In [56]:
get_response(TEXT, ft_index, modelFT, index_map)

['Председатель Банка России Эльвира Набиуллина считает  что ближайшие три года будут трудными для российской экономики  сообщает ТАСС  По ее словам  она пока не ждет существенной перестройки экономики и увеличения ее потенциала  Потенциальные темпы роста оцениваются ЦБ ниже двух процентов  если не будут осуществлены структурные изменения  Набиуллина отметила  что самый важный итог адаптации экономики к новым условиям   это то  что она начинает нащупывать новую модель развития   Задача ЦБ здесь   поддержать этот процесс  обеспечить стабильные финансовые условия для формирования внутренних источников инвестиций     сказала глава регулятора  Набиуллина отметила  что восстановление экономики России может быть растянуто по времени в отличие от кризиса 2008 2009 годов   К настоящему времени наметились признаки восстановления экономики  и в следующем году Банк России ожидает небольшой прирост ВВП  но мы видим  что процессы в разных отраслях и разных регионах очень неустойчивы     добавила она